In [15]:
import numpy as np
import pandas as pd
import re

In [16]:
'''
Parts of the Data processing and EDA for the Titanic Dataset borrowed from:
Niklas Donges
Predicting the Survival of Titanic Passengers
https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8
''' 

'\nParts of the Data processing and EDA for the Titanic Dataset borrowed from:\nNiklas Donges\nPredicting the Survival of Titanic Passengers\nhttps://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8\n'

In [17]:
# import data
df_train = pd.read_csv('../app/data/train.csv')

In [18]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
#Drop id as it is not important
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket'], axis=1).copy()

In [20]:
# Change Cabin to Deck 
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [df_train]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
# we can now drop the cabin feature
df_train = df_train.drop(['Cabin'], axis=1)

In [21]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,0,3,male,22.0,1,0,7.2500,S,8
1,1,1,female,38.0,1,0,71.2833,C,3
2,1,3,female,26.0,0,0,7.9250,S,8
3,1,1,female,35.0,1,0,53.1000,S,3
4,0,3,male,35.0,0,0,8.0500,S,8


In [22]:
#Change Age to Categorical
data = [df_train]

for dataset in data:
    mean = df_train["Age"].mean()
    std = df_train["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = df_train["Age"].astype(int)
df_train["Age"].isnull().sum()

0

In [23]:
#Fill missing with most common
common_value = 'S'
df_train['Embarked'] = df_train['Embarked'].fillna(common_value)

In [24]:
#Change float to int
df_train['Fare'] = df_train['Fare'].astype(int)

In [25]:
#Change genders
genders = {"male": 0, "female": 1}

df_train['Sex'] = df_train['Sex'].map(genders)

In [26]:
#Map and change the ports to numeric
ports = {"S": 0, "C": 1, "Q": 2}
df_train['Embarked'] = df_train['Embarked'].map(ports)

In [27]:
#Group Age into buckets
data = [df_train]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

In [28]:
#Same with fare
for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)

In [29]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,0,3,0,2,1,0,0,0,8
1,1,1,1,5,1,0,3,1,3
2,1,3,1,3,0,0,0,0,8
3,1,1,1,5,1,0,3,0,3
4,0,3,0,5,0,0,1,0,8


In [30]:
"""
Now that the data has been preprocessed it is ready to be used to train the model
I will keep the very last entry as a test for the API call and the rest will be used 
to train the model with.

I am not looking for accuracy here, but more of a proof of concept for a model in 
a docker container that makes prediction based on API calls.  Therfor I will not
be testing the models error in this project.
"""

'\nNow that the data has been preprocessed it is ready to be used to train the model\nI will keep the very last entry as a test for the API call and the rest will be used \nto train the model with.\n\nI am not looking for accuracy here, but more of a proof of concept for a model in \na docker container that makes prediction based on API calls.  Therfor I will not\nbe testing the models error in this project.\n'

In [43]:
df_train.iloc[:-1,:].to_csv('../app/data/prep_train.csv', index=False)
df_train.iloc[-1:,:].to_csv('../app/data/api_test.csv', index=False)